In [1]:
#import libraries
import numpy as np
import pandas as pd
import os
from datetime import time

In [2]:
# join the raw file folder
data_dir = os.path.join(os.getcwd(), "parquet_files")

In [ ]:
# load the file
def load_data(filename):
  return pd.read_parquet(filename)

# create volume moving average
def create_volume_moving_average(df, candles = 8):
  df = df.copy()
  df[f"vma_{candles}"] = df["volume"].rolling(window=candles).mean()  
  return df

# resample logic
def resample_ohlcv(df, timeframe):
  return (df.resample(timeframe, label="right", closed="right").agg({
            "open": "first",
            "high": "max",
            "low": "min",
            "close": "last",
            "volume": "sum",
            "open_interest": "last"
        })
        .dropna(subset=["open"]))

# candle colour logic
def candlestick_colours(df):
  df = df.copy()
  df["candle"] = np.where(
    df["open"]>df["close"],"red", 
    np.where(df["open"]==df["close"],"white","green")
  )
  return df






  
# post processing logic
def post_processing(df): 
  df = df.copy() 
  completed_trade = df['exit_time'].notna()
  df.loc[completed_trade,"pnl_points"] = (df.loc[completed_trade,"exit_price"] - df.loc[completed_trade,"entry_price"])* df.loc[completed_trade,"signal"] 
  df.loc[completed_trade,"monetary_pnl"] = df.loc[completed_trade,"pnl_points"] * df.loc[completed_trade,"quantity"] 
  df['net_pnl'] = df['monetary_pnl'] - df['total_costs']
  df["equity"] = df["net_pnl"].fillna(0).cumsum() 
  df["holding_time"] = df["exit_time"] - df["entry_time"]
  df["trade_direction"] = np.where(df["signal"]==1,"buy",np.where(df["signal"]==-1,"sell",""))
  df['drawdown'] = df['equity'].cummax() - df['equity']
  return df


# final results logic
def result(pnl):
  pnl = pnl.dropna()
  pnl = pnl[pnl!=0]
  wins = pnl[pnl>0]
  losses = pnl[pnl<0]
  avg_win = wins.mean()
  avg_loss = losses.mean()
  win_rate = len(wins)/len(pnl)
  loss_rate = len(losses)/len(pnl)
  expectancy = avg_win*win_rate+avg_loss*loss_rate
  drawdown = max(pnl.fillna(0).cumsum().cummax() - pnl.fillna(0).cumsum()) 
  return {
    "avg_win": avg_win,
    "avg_loss": avg_loss,
    "win_rate": win_rate,
    "loss_rate": loss_rate,
    "expectancy": expectancy,
    "drawdown": drawdown
  } 
  
# trades dataframe logic
def trades_df(df):
  df =df.copy()
  trades_df = df[['trade_direction','quantity','entry_time','exit_time','stop_loss','take_profit','risk_in_points','monetary_pnl','total_costs', 'net_pnl', 'holding_time','equity','drawdown']]
  trades_df = trades_df[trades_df['exit_time'].notna()]
  return trades_df

In [4]:
first_test = os.path.join(data_dir, "RELIANCE.parquet")
reliance = load_data(first_test)
reliance = resample_ohlcv(reliance, "5min")
reliance = create_volume_moving_average(reliance, 8)
reliance = candlestick_colours(reliance)
reliance = reversal(reliance) 
reliance = engulfing(reliance)
reliance = volume_check(reliance, candles = 8)
reliance = signal(reliance)
reliance = entry(reliance)
reliance = stop_loss(reliance)
reliance = risk_in_points(reliance)
reliance = take_profit(reliance, rr = 2)
reliance = calc_quantity(reliance, monetary_risk=1000)
reliance = simulate_trade(reliance)
reliance = transaction_costs(reliance)
reliance = post_processing(reliance)